In [ ]:
#-*- coding: utf-8 -*-
# 라이브러리 import

import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

import datetime
from dateutil.relativedelta import relativedelta
try:
    from urllib.parse import urlencode
except:
    !pip install urllib3
    from urllib.parse import urlencode

import time
import calendar

try:
    import csv
except:
    !pip install csv
    import csv


In [ ]:
# 브라우저 옵션
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument(
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        )
options.add_argument("lang=ko_KR")

In [ ]:
# 문자열 리스트에 문자 존재 반환 함수
def listIndexOf(list, str):
    list_index = 1
    try:
        list_index = list.index(str)
    except ValueError:
        list_index = -1
    return list_index

In [ ]:
# 루프 제한 값
sleepMaxCount = 5
# select query
link_selecter = "a.a.css-4rbku5.css-18t94o4.css-901oao.r-m0bqgq.r-1loqt21.r-1q142lx.r-1qd0xha.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0"
time_selecter = "a.css-4rbku5.css-18t94o4.css-901oao.r-m0bqgq.r-1loqt21.r-1q142lx.r-1qd0xha.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0 > time"
name_selecter = "a.css-4rbku5.css-18t94o4.css-1dbjc4n.r-1loqt21.r-1wbh5a2.r-dnmrzs.r-1ny4l3l > div > div > div > span > span"
id_selecter = "a.css-4rbku5.css-18t94o4.css-1dbjc4n.r-1loqt21.r-1wbh5a2.r-dnmrzs.r-1ny4l3l > div > div.css-1dbjc4n.r-18u37iz.r-1wbh5a2.r-13hce6t > div > span"
content_selecter = "div.css-1dbjc4n > div.css-901oao.r-18jsvk2.r-1qd0xha.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-bnwqim.r-qvutc0"

In [ ]:
# 데이터 추출 함수
def searchWeb(chrome, cvsFileName, search_encode, start_date, end_date, encode_type) :
    chrome.get("https://twitter.com/search?f=live&" + search_encode + "%20until%3A" + end_date + "%20since%3A" + start_date + "&src=typed_query")
    item_query = "section.css-1dbjc4n > div.css-1dbjc4n > div > div"

    i=0
    dataRow=0
    isEnd=0
    time.sleep(5)
    item_len = 0
    link_hrefs = []
    contants_txts = []
    new_items = []
    while item_len==0 and isEnd < sleepMaxCount:
        time.sleep(1)
        items = chrome.find_elements_by_css_selector(item_query)
        item_len = items.__len__()
        isEnd = isEnd + 1
    
    print("스크롤 : ", i)
    new_items = items
    number = 0
    with open(cvsFileName, "a", newline="", encoding=encode_type) as f:
        w = csv.writer(f)
        while new_items.__len__() > 0:
            for item in new_items:
                try:
                    number=number+1
                    links = item.find_elements_by_css_selector(link_selecter)
                    if links.__len__() > 0:
                        link_obj = list(links)[0]
                        href = link_obj.get_attribute("href")
                        if i == 0:
                            link_hrefs.append(href)
                    times = item.find_elements_by_css_selector(time_selecter)
                    time_txt = ""
                    if times.__len__() > 0:
                        time_obj = list(times)[0]
                        time_txt = time_obj.get_attribute("datetime")
                    names = item.find_elements_by_css_selector(name_selecter)
                    name_txt = ""
                    if names.__len__() > 0:
                        name_obj = list(names)[0]
                        name_txt = name_obj.text

                    ids = item.find_elements_by_css_selector(id_selecter)
                    id_txt = ""
                    if ids.__len__() > 0:
                        id_obj = list(ids)[0]
                        id_txt = id_obj.text
                    contents = item.find_elements_by_css_selector(content_selecter)
                    contents_txt = ""
                    contents_urls = ""
                    if contents.__len__() > 0:
                        contents_obj = list(contents)[0]
                        contents_txt = contents_obj.text
                        contents_url_list = re.findall("https?://[^\s]+", contents_txt)
                        if contents_url_list.__len__() > 0:
                            for contents_url in contents_url_list:
                                if contents_url.find("https://twitter.com") == -1:
                                    if contents_urls != "":
                                        contents_urls += ","
                                    contents_urls += contents_url

                    if encode_type == "euc_kr":
                        time_txt = time_txt.encode('euc_kr','replace').decode('euc_kr')
                        name_txt = name_txt.encode('euc_kr','replace').decode('euc_kr')
                        id_txt = id_txt.encode('euc_kr','replace').decode('euc_kr')
                        contents_txt = contents_txt.encode('euc_kr','replace').decode('euc_kr')
                    data = [number, time_txt, name_txt, id_txt, contents_txt, contents_urls]
                    w.writerow(data)
                    #print(number, " => name : ", name_txt, ", id : ", id_txt)
                    print(data)
                except ValueError:
                    print("ValueError")
               
            # 스크롤 맨 아래로 이동
            chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            i = i+1
            print("스크롤 : ", i)
            time.sleep(5)

            # 중복 되지 않은 데이터만 new_items에 담는다.
            new_items = []
            items = chrome.find_elements_by_css_selector(item_query)
            for item in items:
                try: 
                    links = item.find_elements_by_css_selector(link_selecter)
                    if links.__len__() > 0:
                        link_obj = list(links)[0]
                        href = link_obj.get_attribute("href")
                        if listIndexOf(link_hrefs, href) == -1:
                            new_items.append(item)
                            link_hrefs.append(href)
                    else:
                        contents = item.find_elements_by_css_selector(content_selecter)
                        if contents.__len__() > 0:
                            contents_obj = list(contents)[0]
                            contents_txt = contents_obj.text
                            if listIndexOf(contants_txts, contents_txt) == -1:
                                new_items.append(item)
                                contants_txts.append(contents_txt)
                except Exception as e:
                    print(e)
 

In [ ]:
# 날짜 혹은 월별로 나누어서 진행 하기위한 함수
def startSearch() : 
    search_txt = input("search : ")
    date_type = input("search date type (m=월, d=일, default: m) : ")

    now_str = datetime.datetime.now().strftime("%Y-%m-%d")
    now_str_arr = now_str.split("-")
    now_time = datetime.date(int(now_str_arr[0]), int(now_str_arr[1]), int(now_str_arr[2]))

    dsdate_str = now_time.strftime("%Y-%m-%d")
    dedate_str = now_time.strftime("%Y-%m-%d")
    if date_type != "d":
        dsdate_str = now_time.strftime("%Y-%m")
        dedate_str = now_time.strftime("%Y-%m")

    input_sdate = input("start date ("+dsdate_str+") : ")
    input_edate = input("end date ("+dedate_str+") : ")

    if input_sdate is None or input_sdate == "":
        input_sdate = dsdate_str

    if input_edate is None or input_edate == "":
        input_edate = dedate_str


    start_time = now_time
    end_time = now_time

    if date_type != "d":
        sdate_arr = input_sdate.split("-")
        edate_arr = input_edate.split("-")
        start_time = datetime.date(int(sdate_arr[0]), int(sdate_arr[1]), 1)
        end_time = datetime.date(int(edate_arr[0]), int(edate_arr[1]), 1) + relativedelta(months=1) - datetime.timedelta(days=1)
    else:
        sdate_arr = input_sdate.split("-")
        edate_arr = input_edate.split("-")
        start_time = datetime.date(int(sdate_arr[0]), int(sdate_arr[1]), int(sdate_arr[2]))
        end_time = datetime.date(int(edate_arr[0]), int(edate_arr[1]), int(edate_arr[2]))

    if end_time > now_time:
        end_time = now_time
        
    sq = {"q": search_txt}
    txt_encode = urlencode(sq)

    run_time = end_time
    ecd_type="utf-8"

    # 초기 파일 생성
    cvsFileName = "twitter_search_" + search_txt + "_" + input_sdate + "_" + input_edate + ".csv"
    with open(cvsFileName, "w", newline="", encoding=ecd_type) as f:
        colnames = ['number', 'time', 'name', 'id', 'contents', 'content_urls']
        w = csv.writer(f)
        w.writerow(colnames)

    print("start_time: ", start_time.strftime("%Y-%m-%d"))
    print("end_time: ", end_time.strftime("%Y-%m-%d"))

    if __name__ == '__main__' or 'google.colab' in sys.modules:
        chrome = webdriver.Chrome('chromedriver', options=options)
        try :
            while_loop = True
            while while_loop :
                if date_type != "d": # 월단위 검색
                    run_time_str = run_time.strftime("%Y-%m")
                    print("검색 월 : " + run_time_str)
                    run_date_sp = run_time_str.split("-")
                    run_start_time = datetime.datetime(int(run_date_sp[0]), int(run_date_sp[1]), 1)
                    run_end_time = datetime.date(int(run_date_sp[0]), int(run_date_sp[1]), 1) + relativedelta(months=1) - datetime.timedelta(days=1)
                    
                    print(chrome, cvsFileName, txt_encode, run_start_time.strftime("%Y-%m-%d"), run_end_time.strftime("%Y-%m-%d"), ecd_type)
                    time.sleep(1)
                    searchWeb(chrome, cvsFileName, txt_encode, run_start_time.strftime("%Y-%m-%d"), run_end_time.strftime("%Y-%m-%d"), ecd_type)
                    run_time = datetime.date(int(run_date_sp[0]), int(run_date_sp[1]), 1) - relativedelta(months=1)
                    
                    if start_time > run_time:
                        while_loop = False
                    
                else:
                    print("검색 날짜 : " + run_time.strftime("%Y-%m-%d"))
                    run_end_time = run_time + datetime.timedelta(days=1)
                    
                    print(chrome, cvsFileName, txt_encode, run_time.strftime("%Y-%m-%d"), run_end_time.strftime("%Y-%m-%d"), ecd_type)
                    time.sleep(1)
                    searchWeb(chrome, cvsFileName, txt_encode, run_time.strftime("%Y-%m-%d"), run_end_time.strftime("%Y-%m-%d"), ecd_type)
                    run_time = run_time - datetime.timedelta(days=1)
                    if start_time > run_time:
                        while_loop = False

        except Exception as e:
            print(e)
        finally:
            if chrome :
                chrome.close()


In [ ]:
startSearch()